<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/Federated_Learning%20/Flower_Course/second_lesson/MNIST_FL_model_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#adding utils2
from google.colab import files
files.upload()

Saving utils2.py to utils2.py


{'utils2.py': b'"""\nUtility functions and classes for Jupyter Notebooks lessons. \n"""\n\nfrom collections import OrderedDict\nfrom typing import List, Tuple, Dict, Optional\nfrom flwr.common import Metrics, NDArrays, Scalar\nimport torch\nimport torch.nn as nn\nfrom torch.utils.data import Subset, DataLoader, random_split\nimport torch.optim as optim\nfrom torchvision import datasets, transforms\nimport matplotlib.pyplot as plt\nfrom sklearn.metrics import confusion_matrix\nimport seaborn as sns\nimport numpy as np\nimport logging\nfrom flwr.common.logger import console_handler, log\nfrom logging import INFO, ERROR\n\n\nclass InfoFilter(logging.Filter):\n    def filter(self, record):\n        return record.levelno == INFO\n\n\nconsole_handler.setLevel(INFO)\nconsole_handler.addFilter(InfoFilter())\n\ntransform = transforms.Compose(\n    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]\n)\n\n# To filter logging coming from the Simulation Engine\n# so it\'s more readable i

In [2]:
#installing flower
!pip install flwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.6/523.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.69.0
    Uninstalling grpcio-1.69.0:
      Successfully uninstalled grpcio-1.69.0
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1


In [3]:
from utils2 import *
import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import ndarrays_to_parameters, Context
from flwr.server import ServerApp, ServerConfig
from flwr.server import ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation


In [4]:
#recreating datasets
trainset = datasets.MNIST(
    "./MNIST_data/", download=True, train=True, transform=transform
)

total_length = len(trainset)
split_size = total_length // 3
torch.manual_seed(42)
part1, part2, part3 = random_split(trainset, [split_size] * 3)

part1 = exclude_digits(part1, excluded_digits=[1, 3, 7])
part2 = exclude_digits(part2, excluded_digits=[2, 5, 8])
part3 = exclude_digits(part3, excluded_digits=[4, 6, 9])

train_sets = [part1, part2, part3]

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 128MB/s]


Extracting ./MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 23.3MB/s]


Extracting ./MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 83.2MB/s]


Extracting ./MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.53MB/s]

Extracting ./MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw



In [5]:
#recreating testsets
testset = datasets.MNIST(
    "./MNIST_data/", download=True, train=False, transform=transform
)
print("Number of examples in `testset`:", len(testset))

testset_137 = include_digits(testset, [1, 3, 7])
testset_258 = include_digits(testset, [2, 5, 8])
testset_469 = include_digits(testset, [4, 6, 9])

Number of examples in `testset`: 10000


In [6]:
#Setter for model parameterers
def set_weights(net, parameters):
  params_dict = zip(net.state.dic().keys(), parameters)
  state_dict = OrderedDict(
      {k: torch.tensor(v) for k, v in params_dict}
  )
  net.load_state_dict(state_dict, strict=True)
#Getter for model parameters
def get_weights(net):
  ndarrays = [
    val.cpu().numpy() for _, val in net.state_dict().items()
  ]
  return ndarrays


Set_weights:</br>
net is a reference to the PyTorch model and parameters is a list o ndarrays. It uses the list of ndarrays to uptade all the items</br>
Get_weights:</br>
net is a reference to the simple PyTorch model. We iterate over the items convert each one into a numpy ndarray (vetorn-dimensional homogêneo) and return a list containing all of those ndarrays.


In [ ]:
class FlowerClient(NumPyClient):
  def __init__(self, net, trainset, testset, device):
    self.net = net
    self.trainset = trainset
    self.testset = testset

  def fit(self, parameters, config):
    set_weight(self.net, parameters)
    train_model(self.net. self.trainset)
    return get_weights(self.net), len(self.trainset), {}

  def evaluate(self, parameters: NDArrays, config: Dict[str, Scalar]):
    set_weights(self.net, parameters)
    loss, accuracy = evaluate_model(self.net, self.testset)
    return loss, len(self.testset), {"accuracy": accuracy}


self = this

In [ ]:
#client function
def client_fn(context: Context) -> Client:
  net = SimpleModel()
  partition_id = int(context.node_config["partition-id"])
  client_train = train_sets[int(partition_id)]
  client_test = testset
  return FlowerClient(net, client_train, client_test).to_client()

Flower calls client_fn whenever it needs an instance of one particular client to call fit or evaluate

In [ ]:
client = ClientApp(client_fn)

ClientApp represents a client in the federated system, it is responsible to comunicate whith the central server and execute local operations

In [ ]:
def evaluate(server_round, parameters, config):
  net = SimpleModel()
  set_weights(net, parameters)

  _, accuracy = evaluate_model(net, testset)
  _, accuracy137 = evaluate_model(net, testset_137)
  _, accuracy258 = evaluate_model(net, testset_258)
  _, accuracy469 = evaluate_model(net, testset_469)

  log(INFO, "test accuracy on all digits: %.4f", accuracy)
  log(INFO, "test accuracy on [1,3,7]: %.4f", accuracy137)
  log(INFO, "test accuracy on [2,5,8]: %.4f", accuracy258)
  log(INFO, "test accuracy on [4,6,9]: %.4f", accuracy469)

  if server_round == 3:
    cm = compute_confusion_matrix(net, testset)
    plot_confusion_matriz(cm, "Final Global Model")

  return loss,


The evaluarte method evaluates the performance of the neural network model using the provided parameters and the test dataset